To train this model, click **Runtime** > **Run all**.

[![GitHub](https://img.shields.io/badge/GitHub-ART-blue?logo=github)](https://github.com/OpenPipe/ART)
[![Discord](https://img.shields.io/badge/Discord-Join-7289da?logo=discord&logoColor=white)](https://discord.gg/zbBHRUpwf4)
[![Docs](https://img.shields.io/badge/Docs-ART-green)](https://docs.art-e.dev/fundamentals/sft-training)

This notebook demonstrates how to fine-tune a model using **supervised fine-tuning (SFT)** with ART. We'll download a real dataset and train from a JSONL file using `train_sft_from_file`.

For distillation (training from a teacher model's outputs), see the [distillation notebook](https://github.com/OpenPipe/ART/blob/main/examples/sft/distillation.ipynb).

Completions and metrics will be logged to [Weights & Biases](https://wandb.ai).

### Installation

In [ ]:
%%capture
!uv pip install openpipe-art==0.5.9 datasets --prerelease allow --no-cache-dir

### Environment Variables

Set your `WANDB_API_KEY` to use the serverless backend. Get one at [wandb.ai](https://wandb.ai/home).

In [ ]:
import os

WANDB_API_KEY = ""  # required
if WANDB_API_KEY:
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY

### Prepare Dataset

SFT training expects a JSONL file where each line has a `messages` array in the [OpenAI chat format](https://platform.openai.com/docs/api-reference/chat). The last message must be from the `assistant` role.

We'll use [HuggingFaceH4/no_robots](https://huggingface.co/datasets/HuggingFaceH4/no_robots), a high-quality dataset of 10k human-written instruction-following examples that already has a `messages` column in the right format.

In [ ]:
import json

from datasets import load_dataset

with open("train.jsonl", "w") as f:
    for row in load_dataset("HuggingFaceH4/no_robots", split="train"):
        f.write(json.dumps({"messages": row["messages"]}) + "\n")

### Training

Use `train_sft_from_file` to train directly from the JSONL file. It handles batching, learning rate scheduling, and logging automatically.

In [ ]:
import art
from art.serverless.backend import ServerlessBackend
from art.utils.sft import train_sft_from_file

backend = ServerlessBackend()
model = art.TrainableModel(
    name="sft-no-robots",
    project="sft-example",
    base_model="Qwen/Qwen3-30B-A3B-Instruct-2507",
)
await model.register(backend)

await train_sft_from_file(
    model=model,
    file_path="train.jsonl",
    epochs=3,
    batch_size=2,
    peak_lr=2e-4,
    schedule_type="cosine",
    verbose=True,
)

print("Training complete!")

### Inference

Try the trained model.

In [ ]:
client = model.openai_client()
completion = await client.chat.completions.create(
    model=model.get_inference_name(),
    messages=[{"role": "user", "content": "Write a haiku about programming."}],
)
print(completion.choices[0].message.content)

---

For more details, see the [SFT Training docs](https://docs.art-e.dev/fundamentals/sft-training). For distillation, see the [distillation notebook](https://github.com/OpenPipe/ART/blob/main/examples/sft/distillation.ipynb). Questions? Join the [Discord](https://discord.gg/zbBHRUpwf4)!